### Домашнее задание # 1
#### Написать функции:

hit_rate_at_k

money_precision_at_k

recall_at_k

money_recall_at_k

map@k

Mean Reciprocal Rank

*NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции (можно не в pyTorch, а в Numpy - в лекции пример массивов)

**average_precision для ранжирования по примеру с картинки лекции


In [6]:
import numpy as np
import math

In [7]:
#Представим, что в магазине всего 10 товаров
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [8]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    
    return hit_rate_at_k

In [9]:
hit_rate(recommended_list, bought_list)

1

In [10]:
hit_rate_at_k(recommended_list, bought_list, k=10)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [11]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    
    return precision

In [12]:
# Добавим цены на рекомендованные товары
prices_for_recommended = [350, 40, 100, 240, 25, 500, 180, 110, 220, 55, 190, 300, 80, 140, 340]

In [13]:
len(recommended_list) == len(prices_for_recommended)

False

In [14]:
def money_precision_at_k(recommended_list, bought_list, prices_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_list = np.array(prices_list)[:k]
    
    # определяем метки купленных товаров среди рекомендованных
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    # рассчитаем revenue от рекомендованных релеватных товаров скалярным произведением
    recommended_relevant_rev = np.dot(flags, prices_list)
    
    precision_at_k = recommended_relevant_rev/sum(prices_list)
    
    return precision_at_k

In [15]:
money_precision_at_k(recommended_list, bought_list, prices_for_recommended, k=5)

0.7814569536423841

In [16]:
precision(recommended_list, bought_list)

0.2

In [17]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [18]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [19]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    # your_code
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    # your_code
    
    return recall

In [20]:
recall(recommended_list, bought_list)

0.5

In [24]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recall_at_k = sum(flags)/len(bought_list)
    
    return recall_at_k

In [25]:
recall_at_k(recommended_list, bought_list, k=7)

0.5

In [26]:
# Добавим цены на купленные товары
prices_for_bought = list(np.random.randint(20,500,len(bought_list)))

In [27]:
# Заменим значения релевантных рекомедованных товаров на уже известные цены этих рекомендованных товаров
for i in np.where(np.isin(bought_list, recommended_list))[0]:  #возвращаем индексы релеватных рекомендованных элементов
    prices_for_bought[i] = prices_for_recommended[i]

In [28]:
prices_for_bought

[26, 261, 100, 240]

In [29]:
prices_for_recommended

[350, 40, 100, 240, 25, 500, 180, 110, 220, 55, 190, 300, 80, 140, 340]

In [30]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k = 5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recommended_relevant_rev = np.dot(flags, prices_recommended)
    
    money_recall_at_k = recommended_relevant_rev/sum(prices_bought)
    
    return money_recall_at_k

In [31]:
money_recall_at_k(recommended_list, bought_list, prices_for_recommended, prices_for_bought, k=5)

0.9409888357256778

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [32]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [33]:
ap_k(recommended_list, bought_list, k=5)

0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [34]:
def map_k(recommended_list, bought_list, k=5):
    
    # your_code
    
    return result

In [35]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]
                      ] # юзер 3

In [36]:
# Используя имеющиеся функции
def map_k(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    result = np.average([ap_k(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return result

In [37]:
map_k(recommended_list_3_users, bought_list_3_users)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [38]:
import torch
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [39]:
ys_true

tensor([1, 2, 0, 1, 4])

In [40]:
# итоговый результат модели - отсортированный ys_pred
ys_pred

tensor([0.4470, 0.4972, 0.6052, 0.5151, 0.2718])

In [41]:
ys_pred = np.array(ys_pred)

In [42]:
ys_pred

array([0.44697267, 0.49715298, 0.6052205 , 0.5151375 , 0.2718122 ],
      dtype=float32)

In [43]:
def dcg_at_k(recommended_list, bought_list, k=5):
    
    flags = np.isin(recommended_list[:k], bought_list)
    
    norm_coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    return np.dot(flags, norm_coeffs) /k


def ndcg_at_k(recommended_list, bought_list, k=5):
    
    ideal_dcg = dcg_at_k(recommended_list, recommended_list[:k], k)
    return dcg_at_k(recommended_list, bought_list, k) / ideal_dcg 

In [44]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.5426898089849397

In [45]:
#  Общая функция для расчета

def ndcg_k(recommended_list, bought_list, k=5):

    flags = np.isin(recommended_list[:k], bought_list)
    
    #рассчитаем DCG    
    coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    dcg = np.dot(flags, coeffs)
    
    #Рассчитаем идеальный DCG
    ideal_dcg = sum([1/(i+1) if i<=2 else 1/math.log2(i+1) for i in range(k)])
    
    ndcg = dcg/ideal_dcg
    
    return ndcg

In [46]:
ndcg_k(recommended_list, bought_list)

0.5426898089849397

#### Mean Reciprocal Rank ( MRR@k )

In [47]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    rec_relevant_indices = np.nonzero(np.isin(recommended_list[:k], bought_list))[0]
    if len(rec_relevant_indices) == 0:
        return 0
    rr = 1/(rec_relevant_indices[0]+1) #т к расчет идет с 1 (в отличии от индексов, которые с нуля)
    return rr

def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    mrr = np.average([reciprocal_rank(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return mrr

In [48]:
reciprocal_rank(recommended_list, bought_list)

1.0

In [49]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]
                      ] # юзер 3

In [50]:
mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users)

0.3333333333333333